In [ ]:
import pandas as pd
import time
import psycopg2
import getpass

from datetime import timedelta, date
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days + 1)):
        yield start_date + timedelta(n)
        
def logTime():
    return time.strftime("%Y-%m-%d@%H:%M:%S", time.localtime())

def dwh_connect(usr, pwd):
    con=psycopg2.connect(dbname= 'dev',
                         host='wr-dwh-production.cpqq8oq4xy5g.eu-west-1.redshift.amazonaws.com', 
                         port= '5439',
                         user= usr,
                         password= pwd)
    cur = con.cursor()
    return con, cur

print("Please enter your credentials for DWH")
usr = input("username:")
pwd = getpass.getpass("password:")

In [ ]:
sqlFile4w = open('daily_churn_4w.sql', 'r')
sqlScript4w = sqlFile4w.read()
sqlFile4w.close()
sqlScript4w = sqlScript4w.replace('\n','')

sqlFile12w = open('daily_churn_12w.sql', 'r')
sqlScript12w = sqlFile12w.read()
sqlFile12w.close()
sqlScript12w = sqlScript12w.replace('\n','')

In [ ]:
### UPDATE 4W WINDOW DATA
print(logTime(), 'starting 4 week update run')

# Get maximum date from daily retention table
con, cur = dwh_connect(usr, pwd)
cur.execute("SELECT MAX(date) FROM user_temp_schema.jrl_daily_retention WHERE window = '4 week'")
maxDate = cur.fetchall()[0][0]
con.close()

# Set maxDate to 01-Jan-2019 if table is empty
maxDate = [date(2019, 1, 1) if maxDate is None else maxDate][0]
print(logTime(), "current maximum date from table: {0}".format(maxDate))

# Delete the last 30 days from the table
print(logTime(), 'dropping last 30 days from table')
con, cur = dwh_connect(usr, pwd)
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur.execute("""
DELETE FROM user_temp_schema.jrl_daily_retention
WHERE window = '4 week' 
AND date >= '{0}'""".format(maxDate - timedelta(days=29)))
con.close()

# Get the new maximum date from daily retention table
con, cur = dwh_connect(usr, pwd)
cur.execute("SELECT MAX(date) FROM user_temp_schema.jrl_daily_retention WHERE window = '4 week'")
maxDate = cur.fetchall()[0][0]
con.close()

# Set maxDate to 01-Jan-2019 if table is empty
maxDate = [date(2019, 1, 1) if maxDate is None else maxDate][0]

# Set start and end dates for next run
start_date = maxDate + timedelta(days=1)
end_date = date.today() - timedelta(days=28)
print(logTime(), 'running scripts for dates {0} to {1}'.format(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")))

# Iterate through dates in range and insert results into table
for single_date in daterange(start_date, end_date):
    dt = single_date.strftime("%Y-%m-%d")
    con, cur = dwh_connect(usr, pwd)
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur.execute(sqlScript4w.format(dt))
    print(logTime(), 'inserted 4w data for', dt)
    con.close()

# Confirm the run is complete
print(logTime(), 'completed 4 week update run')

In [ ]:
### UPDATE 12W WINDOW DATA
print(logTime(), 'starting 12 week update run')

# Get maximum date from daily retention table
con, cur = dwh_connect(usr, pwd)
cur.execute("SELECT MAX(date) FROM user_temp_schema.jrl_daily_retention WHERE window = '12 week'")
maxDate = cur.fetchall()[0][0]
con.close()

# Set maxDate to 01-Jan-2019 if table is empty
maxDate = [date(2019, 1, 1) if maxDate is None else maxDate][0]
print(logTime(), "current maximum date from table: {0}".format(maxDate))

# Delete the last 30 days from the table
print(logTime(), 'dropping last 30 days from table')
con, cur = dwh_connect(usr, pwd)
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cur.execute("""
DELETE FROM user_temp_schema.jrl_daily_retention
WHERE window = '12 week' 
AND date >= '{0}'""".format(maxDate - timedelta(days=29)))
con.close()

# Get the new maximum date from daily retention table
con, cur = dwh_connect(usr, pwd)
cur.execute("SELECT MAX(date) FROM user_temp_schema.jrl_daily_retention WHERE window = '12 week'")
maxDate = cur.fetchall()[0][0]
con.close()

# Set maxDate to 01-Jan-2019 if table is empty
maxDate = [date(2019, 1, 1) if maxDate is None else maxDate][0]

# Set start and end dates for next run
start_date = maxDate + timedelta(days=1)
end_date = date.today() - timedelta(days=84)
print(logTime(), 'running scripts for dates {0} to {1}'.format(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")))

# Iterate through dates in range and insert results into table
for single_date in daterange(start_date, end_date):
    dt = single_date.strftime("%Y-%m-%d")
    con, cur = dwh_connect(usr, pwd)
    con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cur.execute(sqlScript12w.format(dt))
    print(logTime(), 'inserted 12w data for', dt)
    con.close()

# Confirm the run is complete
print(logTime(), 'completed 12 week update run')